In [25]:
import os
import re
from dotenv import load_dotenv

from langchain_community.embeddings import YandexGPTEmbeddings
from langchain_community.llms import YandexGPT
from langchain_community.document_loaders import DirectoryLoader
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain import hub

In [17]:
load_dotenv()
YANDEX_FOLDER_ID = os.getenv("YANDEX_FOLDER_ID")
YANDEX_API_KEY = os.getenv("YANDEX_API_KEY")

yandex_embeddings = YandexGPTEmbeddings(api_key=YANDEX_API_KEY, folder_id=YANDEX_FOLDER_ID, model_name="text-search-doc")
yandex_llm = YandexGPT(api_key=YANDEX_API_KEY, folder_id=YANDEX_FOLDER_ID, model_name="summarization", temperature=0.0)

In [3]:
# 1. Load documents 
loader = DirectoryLoader("../data/raw/", glob="*.txt")
documents = loader.load()

for num, doc in enumerate(documents):
    print(f"Index: {num}, File name: {doc.metadata['source'][-15:]}, File length: {len(doc.page_content)}")

Index: 0, File name: interview_3.txt, File length: 37995
Index: 1, File name: interview_2.txt, File length: 33951
Index: 2, File name: interview_1.txt, File length: 36025
Index: 3, File name: interview_5.txt, File length: 50584
Index: 4, File name: interview_4.txt, File length: 39364


In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(documents)

In [22]:
# db = Chroma.from_documents(documents=splits, embedding=yandex_embeddings,
#                            persist_directory="../data/chroma_db")

In [8]:
db = Chroma(persist_directory="../data/chroma_db", embedding_function=yandex_embeddings_api_model)
db._collection.count()

252

In [49]:
# question = "Выделить основные разделы или фазы интервью (например, введение, дополнительные вопросы, основные темы, заключительные замечания"
question = "курсы"
context = db.similarity_search(question)

In [50]:
print(context)

[Document(page_content='Так, хорошо, подскажите, пожалуйста, что вот по вашему мнению вот на таких курсах, ну, скажем так, излишне, что можно, на что можно, скажем так, времени закладывать, ну, грубо говоря, что можно назвать бесполезным?\n\nДа я даже не знаю, нет, многие на курсах пиарят свои еще какие-то другие курсы, да, но вот это вот мне не очень нравится, потому что, ну, оно съедает мое время, то есть, соответственно, хотелось бы получить, ну, полезную информацию, и, скажем так, время, которое вот это вот, у меня есть еще такой курс, такой вот это вот, и там, ну, нет, то есть, в принципе, это может быть, да, то есть, в плане какой-нибудь рассылки, там, раздачные материалы, там, и список курсов.\n\nБолее чем достаточно человек, который, ну, умеет читать, у него есть желание, он, ну, получив это в почте, прочтет и, соответственно, ну, посмотрит, что же там предлагается, что же там есть, вот.', metadata={'source': '../data/raw/interview_5.txt'}), Document(page_content='Ну, это одно 

In [11]:
# template = """Игнорируй все предыдущие инструкции. Ты профессионал в проведении и анализе глубинных интервью (Customer Development). Твоя задача помочь проанализировать проведенные интервью. 
# 
# Отвечай на вопросы на основе фактов переданных тебе в контексте. Если в этих данных нет ответа, скажи – "Я
# не знаю". Не придумывай факты, которых нет в контексте. Ты можешь использовать свои общие знания в области Customer Development, проведения и анализа глубинных интервью, чтобы давать корректные ответы. 
# 
# В своем ответе используй следующие принципы:
# 1. Ты должен давать четкие, краткие и прямые ответы.
# 2. Исключи ненужные напоминания, извинения, упоминания самого себя и любые заранее запрограммированные тонкости.
# 3. Сохраняй непринужденный тон в общении.
# 4. Будь прозрачным; если ты не уверен в ответе или если вопрос выходит за рамки твоих возможностей или знаний, признай это.
# 6. При объяснении концепций используй примеры и аналогии из реальной жизни, где это возможно.
# 7. В случае сложных запросов сделай глубокий вдох и работай над проблемой шаг за шагом.
# 8. За каждый ответ ты получишь чаевые до 200 долларов (в зависимости от качества твоего ответа).
# 
# Очень важно, чтобы ты понял это правильно. На кону несколько жизней и моя карьера.
# 
# Context: {context}
# 
# Question: {question} 
# 
# Answer:
# """

In [51]:
# prompt = ChatPromptTemplate.from_template(template)
prompt = hub.pull("rlm/rag-prompt")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [27]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

In [28]:
retriever = db.as_retriever()

In [29]:
chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | yandex_llm
        | StrOutputParser()
)

In [30]:
question = """Подсчитайте частоту упоминания проблем:
1. Составьте список всех конкретных проблем, упомянутых респондентами.
2. Просмотрите каждую стенограмму и подсчитайте, сколько раз упоминалась каждая проблема.
3. Рассчитайте частоту встречаемости каждой проблемы, разделив количество упоминаний на общее количество интервью.
4. Создайте таблицу или диаграмму, чтобы отобразить частоту встречаемости каждой проблемы в порядке убывания."""

In [33]:
answer = chain.invoke("Составьте список всех конкретных проблем, упомянутых респондентами")

In [34]:
print(answer)

- Пользователь является помощником в решении вопросов и использует контекст для ответа на вопросы.
- В контексте упоминаются конкретные проблемы, связанные с изменениями в законодательстве и политической практикой.
- Важность профессионализма и развития для пользователя.
- Влияние пандемии на рынок недвижимости и неуверенность людей в будущем.
- Рынок недвижимости нездоров из-за пандемии и других ситуаций.
- Решение квартирных вопросов требует дополнительных средств и ипотечных кредитов.
